# Libraries, Packages and Functions

In [1]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 2.8MB/s a 0:00:011


In [5]:
!pip install geopy

    100% |████████████████████████████████| 112kB 2.4MB/s ta 0:00:01


In [7]:
import os
import tarfile
import json
import pandas as pd
import numpy as np
#from google.colab import drive
from geopy import Nominatim

In [8]:
geolocator = Nominatim(user_agent="geoapiExercises")

def city_state_country(coord):
  try:
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    #city = address.get('city', '')
    state = address.get('state', '')
    #country = address.get('country', '')
    return state
  except:
    return 'NULL'
#print(city_state_country("47.470706, -99.704723"))

# 1) Loading source File

In [9]:
#Set WD
# Use this line in Colab
#drive.mount("/content/drive")
#Use this line in local jupyter
os.chdir("/Users/andreaprenner/Desktop/Master_UvA/1. Semester/FundamentalsOfDS/Assignment1")

In [10]:
# Use this line in Colab
#tar = tarfile.open('/content/drive/My Drive/Colab Notebooks/Assignment 1 FDS/geotagged_tweets_20160812-0912.tar')
# Use this line in local jupyter
tar = tarfile.open("geotagged_tweets_20160812-0912.tar.gz")
files = tar.getmembers()

In [11]:
f = tar.extractfile(files[0]) 
data = f.readlines()

In [ ]:
#One tweet entry in raw data file
json.loads(data[0])

In [12]:
len(data)

657307

In [ ]:
for i in range(0, 10000): 
  dic = json.loads(data[i*6]) #Why i times 6?
  
  try:
    # first iteration we create the dataframe
    if i == 0:
      base_data = {'created_at': [dic['created_at']]
                  ,'text': [dic['text']]
                  ,'id': [dic['id']]
                  ,'tweet_lang': [dic['lang']]
                  ,'in_reply_to_user_id': [dic['in_reply_to_user_id']]
                  ,'in_reply_to_user_id_str': [dic['in_reply_to_user_id_str']]
                  ,'user_id': [dic['user']['id']]
                  ,'user_screen_name' : [dic['user']['screen_name']]
                  ,'user_location': [dic['user']['location']]
                  ,'user_description': [dic['user']['description']]
                  ,'user_verified': [dic['user']['verified']]
                  ,'user_followers_count': [dic['user']['followers_count']]
                  ,'user_friends_count': [dic['user']['friends_count']]
                  ,'user_statuses_count': [dic['user']['statuses_count']]
                  ,'user_created_at': [dic['user']['created_at']]
                  ,'user_lang': [dic['user']['lang']]
                  #,'place_id' : [dic['place']['id']]
                  ,'place_name' : [dic['place']['name']]
                  ,'place_full_name' : [dic['place']['full_name']]
                  ,'place_country_code' : [dic['place']['country_code']]
                  ,'place_country' : [dic['place']['country']]
                  ,'place_latitude' : [dic['place']['bounding_box']['coordinates'][0][0][0]]
                  ,'place_longitude' : [dic['place']['bounding_box']['coordinates'][0][0][1]]
                  ,'entities_hashtags' : [dic['entities']['hashtags']]
                  ,'entities_user_mentions' : [dic['entities']['user_mentions']]
                  }
      df_base = pd.DataFrame(data=base_data)
    
    # next iterations we append data
    else:
      next_data = {'created_at': [dic['created_at']]
                  ,'text': [dic['text']]
                  ,'id': [dic['id']]
                  ,'tweet_lang': [dic['lang']]
                  ,'in_reply_to_user_id': [dic['in_reply_to_user_id']]
                  ,'in_reply_to_user_id_str': [dic['in_reply_to_user_id_str']]
                  ,'user_id': [dic['user']['id']]
                  ,'user_screen_name' : [dic['user']['screen_name']]
                  ,'user_location': [dic['user']['location']]
                  ,'user_description': [dic['user']['description']]
                  ,'user_verified': [dic['user']['verified']]
                  ,'user_followers_count': [dic['user']['followers_count']]
                  ,'user_friends_count': [dic['user']['friends_count']]
                  ,'user_statuses_count': [dic['user']['statuses_count']]
                  ,'user_created_at': [dic['user']['created_at']]
                  ,'user_lang': [dic['user']['lang']]
                  #,'place_id' : [dic['place']['id']]
                  ,'place_name' : [dic['place']['name']]
                  ,'place_full_name' : [dic['place']['full_name']]
                  ,'place_country_code' : [dic['place']['country_code']]
                  ,'place_country' : [dic['place']['country']]
                  ,'place_latitude' : [dic['place']['bounding_box']['coordinates'][0][0][0]]
                  ,'place_longitude' : [dic['place']['bounding_box']['coordinates'][0][0][1]]
                  ,'entities_hashtags' : [dic['entities']['hashtags']]
                  ,'entities_user_mentions' : [dic['entities']['user_mentions']]
                  }            
      df_aux = pd.DataFrame(data=next_data)
      df_base = df_base.append(df_aux)
  except:
    print("Error")

In [ ]:
data_formatted = df_base

In [ ]:
# Save Data
#data_formatted.to_pickle('/content/drive/My Drive/Colab Notebooks/Assignment 1 FDS/data_formatted.pkl')
data_formatted = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/Assignment 1 FDS/data_formatted.pkl")
data_formatted

# 2) Cleaning, Filtering and Formatting

## 2.1) Only US, English Languague

In [ ]:
#Exclude tweets outside of US (location) - filter on country code US (Country where tweet is posted)
data_formatted = data_formatted[data_formatted['place_country_code'] == "US"]
data_formatted = data_formatted[data_formatted['tweet_lang'] == 'en']
len(data_formatted)

## 2.2) State

In [ ]:
data_formatted['coordinates'] = data_formatted['place_longitude'].astype(str) + ", " + data_formatted['place_latitude'].astype(str)
data_formatted['state'] = data_formatted['coordinates'].apply(city_state_country)
#data_formatted

In [ ]:
data_formatted.to_pickle('/content/drive/My Drive/Colab Notebooks/Assignment 1 FDS/data_formatted.pkl')

## 2.3) Hash/Tags

In [ ]:
#Determine tags and hashs (some tweets dont have tags only hash)
data_formatted['tags_mention'] = data_formatted.text.str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join)
data_formatted['hash_mention'] = data_formatted.text.str.findall(r"#(\w+)").apply(','.join)
data_formatted['tags_hash'] = data_formatted['hash_mention'] + data_formatted['tags_mention']

In [ ]:
#Lower cases in tags and hashes
data_formatted['tags_hash'] = data_formatted['tags_hash'].apply(lambda x: x.lower())

In [ ]:
#See if either trum or clinton mentioned in hash or tags
data_formatted['Trump'] = data_formatted['tags_hash'].str.contains(r'trump|maga', na=False)
data_formatted['Clinton'] = data_formatted['tags_hash'].str.contains(r'clinton|imwithher|hillary', na=False)

In [ ]:
data_formatted.to_pickle('/content/drive/My Drive/Colab Notebooks/Assignment 1 FDS/data_formatted.pkl')

## 2.4) Filtering Bots

In [ ]:
#Filter bots (hyperlinks in text)
#Removing dubious sources: 
sources = ('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>','<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
                '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', '<a href="https://twitter.com/download/android" rel="nofollow">Twitter for Android Tablets</a>',
                '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>','<a href="http://instagram.com" rel="nofollow">Instagram</a>')

data_formatted =data_formatted[data_formatted.source.isin(sources)]

#Removing people with no friends and no followers 
data_formatted = data_formatted[(data_formatted.followers_count > 0) & (data_formatted.followers_count > 0)]

In [ ]:
data_formatted.to_pickle('/content/drive/My Drive/Colab Notebooks/Assignment 1 FDS/data_formatted.pkl')

## 2.5) More Columns - Same as 2.3??

In [ ]:
#Determine tags and hashs (some tweets dont have tags only hash)
data_formatted['tags_mention'] = data_formatted.text.str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join)
data_formatted['hash_mention'] = data_formatted.text.str.findall(r"#(\w+)").apply(','.join)
data_formatted['tags_hash'] = data_formatted['hash_mention'] + data_formatted['tags_mention']

In [ ]:
#Lower cases in tags and hashes
data_formatted['tags_hash'] = data_formatted['tags_hash'].apply(lambda x: x.lower())

In [ ]:
#See if either trum or clinton mentioned in hash or tags
data_formatted['Trump'] = data_formatted['tags_hash'].str.contains(r'trump', na=False)
data_formatted['Clinton'] = data_formatted['tags_hash'].str.contains(r'clinton', na=False)

In [ ]:
data_formatted.to_pickle('/content/drive/My Drive/Colab Notebooks/Assignment 1 FDS/data_formatted.pkl')

# 3) Exploratory Analysis

In [ ]:
#data_("How many different ids we have : " + str(data_formatted3['id'].nunique()))
#print("How many different ids we have : " + str(data_formatted['location'].unique

# 4) Sentiment Analysis

In [ ]:
import nltk
from nltk.corpus import stopwords
import random
import re

In [ ]:
#Loading training dataset 
training_set = pd.read_csv("trainingandtestdata/training.1600000.processed.noemoticon.csv", 
                 encoding = 'latin-1',
                 names = ["Polarity", "Tweet ID","Date","Query","User","Text"])

In [ ]:
#Define english stopwords
stopwords = stopwords.words('english')

In [ ]:
#Filter out Polarity Score equal 0 or 4
training_set = training_set[(training_set["Polarity"] == 4) | (training_set["Polarity"] == 0)]

In [ ]:
#Data cleaning - removing punctuation from text; removing tags; removing links
training_set['Text'] = training_set['Text'].str.replace('@[^\s]+','')
training_set['Text'] = training_set['Text'].str.replace('[^\w\s]','')
training_set['Text'] = training_set['Text'].str.replace('\[.*?\]','')
training_set['Text'] = training_set['Text'].str.replace('[‘’“”…]','')
training_set['Text'] = training_set['Text'].str.replace('\w*\d\w*','')

In [ ]:
#Create data that puts tweet texts together with sentiment score
tweets_and_polarity_scores = []
for i in range(0, len(training_set)):
    if training_set["Polarity"].iloc[i] == 0 or training_set["Polarity"].iloc[i] == 4: 
        tweet_text = training_set['Text'].iloc[i]
        tweet_words_in_list = [e.lower() for e in tweet_text.split()]
        tweet_words_in_list_wo_stopwords = [e for e in tweet_words_in_list if not e in stopwords]
        tweet_polarity_score = training_set["Polarity"].iloc[i]
        tweets_and_polarity_scores.append((tweet_words_in_list_wo_stopwords, tweet_polarity_score))
    else: 
        continue

In [ ]:
#Select random sample - otherwise the classifier will be too slow
random_sample = random.sample(tweets_and_polarity_scores, k = 10000)

In [ ]:
def get_words_in_tweets(tweets):
    all_words= []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = list(wordlist.keys())
    return word_features

word_features = get_word_features(get_words_in_tweets(random_sample))

In [ ]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features["contains(%s)" % word] = (word in document_words)
    return features

In [ ]:
final_training_set = nltk.classify.apply_features(extract_features, random_sample)

In [ ]:
#Classifier way too slow atm
classifier = nltk.NaiveBayesClassifier.train(final_training_set)

In [ ]:
#Testing
tweet_positive = 'I like Larry'
classifier.classify(extract_features(tweet_positive.split()))
#classifier.show_most_informative_features(32)

4

In [ ]:
#Loading test dataset 
test_set = pd.read_csv("trainingandtestdata/testdata.manual.2009.06.14.csv", 
                 encoding = 'latin-1',
                 names = ["Polarity", "Tweet ID","Date","Query","User","Text"])

In [ ]:
#Data cleaning - removing punctuation from text
test_set['Text'] = test_set['Text'].str.replace('@[^\s]+','')
test_set['Text'] = test_set['Text'].str.replace('[^\w\s]','')
test_set['Text'] = test_set['Text'].str.replace('\[.*?\]','')
test_set['Text'] = test_set['Text'].str.replace('[‘’“”…]','')
test_set['Text'] = test_set['Text'].str.replace('\w*\d\w*','')

In [ ]:
#Split words in tweets; remove stop words
test_set['Text'] = test_set['Text'].apply(lambda x: x.lower())
test_set['Text'] = test_set['Text'].apply(lambda x: x.split())
test_set['Text'] = test_set['Text'].apply(lambda x: [item for item in x if item not in stop])

In [ ]:
#Apply classifier to column 
test_set['classification_polarity'] = test_set['Text'].apply(lambda x: classifier.classify(extract_features(x)))

In [ ]:
test_set.head()

,Polarity,Tweet ID,Date,Query,User,Text,classification_polarity
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,"[loooooooovvvvvveee, dx, cool, fantastic, right]",4
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,"[reading, love, lee, childs, good, read]",4
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"[ok, first, assesment, fucking, rocks]",4
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,"[youll, love, ive, mine, months, never, looked...",0
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,"[fair, enough, think, perfect]",4


# 5) Topic Modelling - LDA

## Using all words

In [ ]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import nltk
nltk.download('wordnet')
import gensim
from gensim import corpora

In [ ]:
data_sentimentanalysis = data_formatted.copy()

In [ ]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [ ]:
stopwords = stopwords.words('english')

In [ ]:
#Data cleaning - removing punctuation from text; still need to remove hashes
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('@[^\s]+','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('[^\w\s]','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('\[.*?\]','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('[‘’“”…]','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('\w*\d\w*','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].apply(lambda x: x.lower())
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].apply(lambda x: x.split())
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].apply(lambda x: [item for item in x if item not in stopwords])
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].apply(lemmatize_text)

In [ ]:
#Create data set for trump vs clinton
trump_mentioned_true = data_sentimentanalysis[data_sentimentanalysis['Trump']]
clinton_mentioned_true = data_sentimentanalysis[data_sentimentanalysis['Clinton']]

In [ ]:
#Filter out rows where text doesnt contain any entries after removing tags and hashes
trump_mentioned_true = trump_mentioned_true[trump_mentioned_true['text'].map(lambda d: len(d)) > 0]
trump_text = trump_mentioned_true.text.tolist()
dictionary_trump = corpora.Dictionary(trump_text)
doc_term_matrix_trump = [dictionary_trump.doc2bow(doc) for doc in trump_text]
# Creating the object for LDA model using gensim library
Lda_trump = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel_trump = Lda_trump(doc_term_matrix_trump, num_topics=2, id2word = dictionary_trump, passes=100)
topics_trump = ldamodel_trump.print_topics(num_topics=2, num_words=8)
topics_trump

In [ ]:
#Filter out rows where text doesnt contain any entries after removing tags and hashes
clinton_mentioned_true = clinton_mentioned_true[clinton_mentioned_true['text'].map(lambda d: len(d)) > 0]
clinton_text = clinton_mentioned_true.text.tolist()
dictionary_clinton = corpora.Dictionary(clinton_text)
doc_term_matrix_clinton = [dictionary_clinton.doc2bow(doc) for doc in clinton_text]
# Creating the object for LDA model using gensim library
Lda_clinton = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel_clinton = Lda_clinton(doc_term_matrix_clinton, num_topics=2, id2word = dictionary_clinton, passes=100)
topics_clinton = ldamodel_clinton.print_topics(num_topics=10, num_words=4)
topics_clinton

## Only using nouns

In [ ]:
from nltk import word_tokenize, pos_tag
import numpy as np

In [ ]:
#Apply function that determines the type of each word
trump_mentioned_text = trump_mentioned_true['text'].apply(lambda x: nltk.pos_tag(x))

In [ ]:
#Filter out the nouns for each row
trump_mentioned_nouns = pd.DataFrame(np.zeros((len(trump_mentioned_true), 1)))
trump_mentioned_nouns.columns = ['Text']
for i in range(0, len(trump_mentioned_text)):
    nouns = [word for (word, pos) in trump_mentioned_text.iloc[i] if is_noun(pos)]
    nouns = ' '.join(nouns)
    trump_mentioned_nouns.iloc[i] = nouns

In [ ]:
#Split the Text 
trump_mentioned_nouns['Text'] = trump_mentioned_nouns['Text'].apply(lambda x: x.split())

In [ ]:
trump_text = trump_mentioned_nouns.Text.tolist()
dictionary_trump = corpora.Dictionary(trump_text)
doc_term_matrix_trump = [dictionary_trump.doc2bow(doc) for doc in trump_text]
Lda_trump = gensim.models.ldamodel.LdaModel
ldamodel_trump = Lda_trump(doc_term_matrix_trump, num_topics=10, id2word = dictionary_trump, passes=100)
topics_trump = ldamodel_trump.print_topics(num_topics=10, num_words=8)
topics_trump

In [ ]:
#Apply function that determines the type of each word
clinton_mentioned_text = trump_mentioned_true['text'].apply(lambda x: nltk.pos_tag(x))

In [ ]:
#Filter out the nouns for each row
clinton_mentioned_nouns = pd.DataFrame(np.zeros((len(clinton_mentioned_true), 1)))
clinton_mentioned_nouns.columns = ['Text']
for i in range(0, len(clinton_mentioned_text)):
    nouns = [word for (word, pos) in clinton_mentioned_nouns.iloc[i] if is_noun(pos)]
    nouns = ' '.join(nouns)
    clinton_mentioned_nouns.iloc[i] = nouns

In [ ]:
#Split the text 
clinton_mentioned_nouns['Text'] = clinton_mentioned_nouns['Text'].apply(lambda x: x.split())

In [ ]:
clinton_text = clinton_mentioned_nouns.text.tolist()
dictionary_clinton = corpora.Dictionary(clinton_text)
doc_term_matrix_clinton = [dictionary_clinton.doc2bow(doc) for doc in clinton_text]
Lda_clinton = gensim.models.ldamodel.LdaModel
ldamodel_clinton = Lda_clinton(doc_term_matrix_clinton, num_topics=10, id2word = dictionary_clinton, passes=100)
topics_clinton = ldamodel_clinton.print_topics(num_topics=10, num_words=8)
topics_clinton

# Others (No need to run this)


In [ ]:
#Excluding tweets referring to Clinton & Trump
#Getting sentiment score on daily basis 

Mapping City to State DF

In [ ]:
#Loading File
locations = pd.read_csv("https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/cities/totals/sub-est2019_all.csv", encoding = 'latin-1')

#Keep Columns we need
locations = locations[['NAME','STNAME']].copy()

#Only cities
locations = locations[locations['NAME'].str.contains("city")]

#Remove "city" word
locations["NAME"] = locations["NAME"].str.replace("city", "")
#locations["NAME"] = locations["NAME"].str.replace("town", "")
#locations["NAME"] = locations["NAME"].str.replace("County", "")
#locations["NAME"] = locations["NAME"].str.replace("Balance of ", "")

#trimming
locations["NAME"] = locations["NAME"].str.strip()

#Unique Cities
locations.drop_duplicates(subset = 'NAME', keep = 'first', inplace = True)
locations


,NAME,STNAME
1,Abbeville,Alabama
2,Adamsville,Alabama
5,Alabaster,Alabama
6,Albertville,Alabama
7,Alexander City,Alabama
...,...,...
81240,Lander,Wyoming
81241,Laramie,Wyoming
81264,Rawlins,Wyoming
81268,Rock Springs,Wyoming


In [ ]:
def city_to_state(city):
    """
    Gets the state of a city
    :param city: the city name for which we are looking the state
    :return data:
    """
    state = locations[locations['NAME'].str.lower().str.contains(city.lower())]['STNAME'].head(1)
    return state

In [ ]:
city_to_state("New York")

33132    Minnesota
Name: STNAME, dtype: object

In [ ]:
city_to_state(data_formatted['location'][0])

5278    Florida
Name: STNAME, dtype: object

In [ ]:
#Change dictionary into dataframe
#Load dictionaries within dictionaries (user, place) separately. From user exclude unnecessary columns
data_formatted = []
for i in range(0, 1000): 
    data_f = json.loads(data[i])
    user = data_f['user']
    user = pd.DataFrame([user])
    unwanted_user = set(['id_str', 'profile_background_color', 'profile_background_image_url', 
                         'profile_background_image_url_https', 'profile_background_tile', 
                        'profile_link_color', 'profile_sidebar_border_color', 
                        'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 
                        'profile_image_url', 'profile_image_url_https', 'default_profile_image']) 
    for unwanted_key in unwanted_user: del user[unwanted_key]
    place = data_f['place']
    place = pd.DataFrame([place])
    
    place = place[['full_name', 'country_code', 'bounding_box']]
    latitude = data_f['place']['bounding_box']['coordinates'][0][0][0]
    longitude = data_f['place']['bounding_box']['coordinates'][0][0][1]
    
    unwanted = set(['user', 'place', 'entities']) #Dictionaries within dictionaries
    for unwanted_key in unwanted: del data_f[unwanted_key]
    data_pre = pd.DataFrame([data_f])
    data_pre['latitude'] = latitude;
    data_pre['longitude'] = longitude;
    user = user.rename({'id':'user_id'}, axis=1)
    if i == 0: 
        data_formatted = data_pre
        user_formatted = user
        place_formatted = place
    else:
        data_formatted = data_formatted.append(data_pre, ignore_index=True)
        user_formatted = user_formatted.append(user, ignore_index=True)
        place_formatted = place_formatted.append(place, ignore_index=True)

In [ ]:
#Merge data frame (without dictionaries within dictionaries), user and place dictionary together
#data_formatted = pd.concat([data_formatted, user_formatted, place_formatted], axis=1)
#data_formatted.head()

ValueError: ignored